### Import knihoven a konfigurace

In [1]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
#from pypylon import pylon 
from IPython.display import clear_output, display

#import pytesseract
from PIL import Image

%matplotlib inline

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

<a id='connect_functions'>Funkce k připojení kamery.</a>

In [2]:
# List of camera features to create widgets
FEATURES = [
    {
        "name": "GainRaw",
        "type": "int"
    },
    {
        "name": "ExposureTimeRaw",
        "type": "int",
        "max": 35000,
        "min": 35,
        "step": 140,
        "value": 30124
    },
    {
        "name": "Height",
        "type": "int_text",
        "max": 1200,
        "min": 100,
        "step": "5"
    },
    {
        "name": "Width",
        "type": "int_text",
        "max": 1920,
        "min": 100,
        "step": "5"
    }
]

FEATURES_PERICENTRIC = [
    {
        "name": "Gain",
        "type": "int"
    },
    {
        "name": "ExposureTime",
        "type": "int",
        "max": 35000,
        "min": 90,
        "step": 30,
        "value": 1500
    },
    {
        "name": "Height",
        "type": "int_text",
        "max": 2000,
        "min": 100,
        "step": "1",
        'value':1270
    },
    {
        "name": "Width",
        "type": "int_text",
        "max": 2000,
        "min": 100,
        "step": "1",
        'value':1344
    },
    {
        "name": "OffsetX",
        "type": "int_text",
        "max": 1000,
        "min": 0,
        "step": "1",
        'value':608
    },
    {
        "name": "OffsetY",
        "type": "int_text",
        "max": 1000,
        "min": 0,
        "step": "1",
        'value':418
    }
]

In [3]:
def connect_camera(serial_number):
    ''' Connects camera specified with its serial number
    
    Parameters
    ----------
    serial_number : string
        Camera's serial number.
    grabbed_images_path : string
        Path to folder where the saved images will be stored.
    Returns
    -------
    camera : object
    '''
    info = None
    for i in pylon.TlFactory.GetInstance().EnumerateDevices():
        if i.GetSerialNumber() == serial_number:
            info = i
            break
    else:
        print('Camera with {} serial number not found'.format(serial_number))

    # VERY IMPORTANT STEP! To use Basler PyPylon OpenCV viewer you have to call .Open() method on you camera
    if info is not None:
        camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateDevice(info)) 
        camera.Open()
        return camera
    else:
        return None    

Funkce pro zobrazení okna s přidáním eventu na klik myší.

In [3]:
def show_camera_window(*imgs, scale=1):
    def print_xy(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONUP:
            print('x = %d, y = %d'% (x, y))  
        
    for i, img in enumerate(imgs, 1):
        window_name_id = 'Camera capture' + ' ' + str(i)
        
        h,w = img.shape[:2]
        cv2.namedWindow(window_name_id, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
        cv2.resizeWindow(window_name_id, int(w * scale), int(h * scale))
        cv2.setMouseCallback(window_name_id, print_xy)
        if len(imgs) > 1:
            cv2.moveWindow(window_name_id, (i-1)*int(w * scale), 0)
        cv2.imshow(window_name_id, img)

<a id='preprocessing_functions'>Metody předzpracování.</a>

In [5]:
def to_gray(img):
    ''' Converts image to monochrome
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    if len(img.shape) == 2:
        return img
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def to_hsv(img):
    ''' Converts image to HSV (hue, saturation, value) color space.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    dst = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    return dst

def negative(img):
    ''' Converts image to its negative
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    dst = 255 - img
    return dst

def crop(img, tl_x, tl_y, br_x, br_y):
    ''' Crops image by added coordinates
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    tl_x : int
        TOP-LEFT corner's x-coordinate
    tl_y : int
        TOP-LEFT corner's y-coordinate
    br_x : int
        BOTTOM-RIGHT corner's x-coordinate
    br_y : int
        BOTTOM-RIGHT corner's y-coordinate
    Returns
    -------
    Ouput image.
    '''
    roi = img[tl_y:br_y, tl_x:br_x]
    return roi    

<a id='segmentation_functions'>Metody segmentace.</a>

In [3]:
def segmentation_one_threshold(img, threshold):
    '''Segments image into black & white using one threshold
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    threshold : int
        Pixels with value lower than threshold are considered black, the others white.
    Returns
    -------
    Ouput image.
    '''
    _, dst = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    return dst

def segmentation_auto_threshold(img):
    '''Segments image into black & white using automatic threshold
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    _, dst = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return dst

def segmentation_two_thresholds(img, lower, higher):
    '''Segments image into black & white using two thresholds
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    lower : int
        Pixels with value lower than threshold are considered black, the others white.
    higher : int
        Pixels with value higher than threshold are considered black, the others white.
    Returns
    -------
    Ouput image.
    '''
    return cv2.inRange(img, min(lower, higher), max(lower, higher))

def segmentation_adaptive_threshold(img, size, constant=0):
    '''Segments image into black & white using calculated adaptive 
    threshold using Gaussian function in pixel neighbourhood.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    size : int
        Size of used gaussian. Lowest value is 3. Algorithm uses only odd numbers.
    constant : int
        Value that is added to calculated threshlod. It could be negative as well as zero as well as positive number.
    Returns
    -------
    Ouput binary image.
    '''
    if size < 3:
        size = 3
    elif size % 2 == 0:
        size -= 1
    return cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, size, int(constant))

def contours(img_bin, min_area=0, max_area=1000000, fill=True):
    '''Finds contours in binary image and filters them using their area. Then it draws binary image
    from filtered contours. It counts contours as well.
    
    Parameters
    ----------
    img_bin : numpy.ndarray
        Input binary image.
    min_area : int
        Size of contour that is used to filter all smaller contours out.
    max_area : int
        Size of contour that is used to filter all larger contours out.
    Returns
    -------
    contour_drawn : numpy.ndarray
        Output binary image with drawn filled filtered contours.
    count : int
        Number of found and filtered contours.
    contours : list
        Found contours.
    '''
    _, contours, _  = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours =  [c for c in contours if cv2.contourArea(c) > min_area and cv2.contourArea(c) < max_area]
    thick = cv2.FILLED
    if not fill: thick = 2
    contour_drawn = cv2.drawContours(np.zeros(img_bin.shape, dtype=np.uint8), contours, -1, color=(255, 255, 255), thickness=thick)
    return contour_drawn, len(contours), contours    

In [3]:
# Dimensionless descriptors
class ShapeDescriptors:
    def form_factor(area, perimeter):
        return (4 * np.pi * area) / (perimeter * perimeter)
    
    def roundness(area, max_diameter):
        return (4 * area) / (np.pi * max_diameter * max_diameter)
    
    def aspect_ratio(min_diameter, max_diameter):
        return min_diameter / max_diameter;
    
    def convexity(perimeter, convex_perimeter):
        return convex_perimeter / perimeter
    
    def solidity(area, convex_area):
        return area / convex_area
    
    def compactness(area, max_diameter):
        return np.sqrt(4 / np.pi * area) / max_diameter;
        
    def extent(area, bounding_rectangle_area):
        return area / bounding_rectangle_area;

# Špičatost
def form_factor(bin_im):
    _, _, conts = contours(bin_im)
    return ShapeDescriptors.form_factor(cv2.contourArea(conts[0]), cv2.arcLength(conts[0], True))

# Kulatost
def roundness(bin_im):
    _, _, conts = contours(bin_im)
    area = cv2.contourArea(conts[0])
    _,radius = cv2.minEnclosingCircle(conts[0])
    r = ShapeDescriptors.roundness(area, 2*radius)
    if r > 1: r = 1
    return r

# Poměr stran
def aspect_ratio(bin_im):
    _, _, conts = contours(bin_im)
    dims = cv2.minAreaRect(conts[0])[1]
    min_diameter = min(dims)
    max_diameter = max(dims)
    return ShapeDescriptors.aspect_ratio(min_diameter, max_diameter)
    
# Konvexita, vypouklost
def convexity(bin_im):
    _, _, conts = contours(bin_im)
    hull = cv2.convexHull(conts[0], None, True, True)
    per = cv2.arcLength(conts[0], True)
    conv_per = cv2.arcLength(hull, True)
    r = ShapeDescriptors.convexity(per, conv_per)
    if r > 1: r = 1
    return r 

# Plnost, celistvost
def solidity(bin_im):
    _, _, conts = contours(bin_im)
    hull = cv2.convexHull(conts[0], None, True, True)
    area = cv2.contourArea(conts[0])
    conv_area = cv2.contourArea(hull)
    r = ShapeDescriptors.solidity(area, conv_area)
    if r > 1: r = 1
    return r 
    
# Kompaktnost, hutnost
def compactness(bin_im):
    _, _, conts = contours(bin_im)
    area = cv2.contourArea(conts[0])
    max_diameter = max(cv2.minAreaRect(conts[0])[1])
    r = ShapeDescriptors.compactness(area, max_diameter)
    if r > 1: r = 1
    return r 
    
# Dosah, rozměrnost
def extent(bin_im):
    _, _, conts = contours(bin_im)
    area = cv2.contourArea(conts[0])
    w, h = cv2.minAreaRect(conts[0])[1]
    return ShapeDescriptors.extent(area, w*h)

<a id='filtration_functions'>Metody filtrace.</a>

In [2]:
def filtration_median(img, filter_size):
    '''Filters image noise using median algorithm
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    filter_size : int
        Size of median filter.
    Returns
    -------
    Ouput image.
    '''
    return cv2.medianBlur(img, filter_size)   

def fill_holes(img_bin, close=False, size=5):
    '''Fill holes in found contours. It could merge the contour using close input with appropriate size.
    
    Parameters
    ----------
    img_bin : numpy.ndarray
        Input binary image.
    close : boolean
        If it should merge contours with missing points using close operation.
    size : int
        Size of close operation element.
    Returns
    -------
    Ouput binary image.
    '''
    if close:
        struct = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size, size))
        img_bin = cv2.morphologyEx(img_bin, cv2.MORPH_CLOSE, struct)
    res, _, _ = contours(img_bin)
    return res

<a id='show_functions'>Metoda pro zobrazení různého množství obrázků.</a>

In [2]:
def select_image_points(img, points_cnt = 4):
    """ Opens a new image window, where user can interactively add or remove image points.
    Points are added while holding CTRL key + pressing left mouse button and removed by ALT key + pressing left mouse button.
    The point selection is terminated by pressing the 'q' key.
    
    Parameters
    ----------
    img : ndarray
        Input image where image points are choosen and drawn.
    points_cnt : Optional[int]
        A maximum number of points to choose. A minimum number of points to compute the projective transformation is 4.
    Returns
    -------
    list
        Returns a list of size >= 4 and size <= points_cnt such that each elements represent (x, y) coordinate in input image.
    """
    if points_cnt < 4: 
        raise ValueError('Number of points must be >= 4.')
        
    points = []
    window_name = 'Point selection'
    img_dimensions = img.shape[:2]
    pts_dist_thresh = 0.01 * img_dimensions[1] # Scale drawing elements with image size
    
    def draw_circle(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            if flags == cv2.EVENT_FLAG_ALTKEY + cv2.EVENT_LBUTTONDOWN: 
                for p in points:
                    if distance.euclidean(p, (x, y)) < pts_dist_thresh:
                        points.remove(p)
                        break
            elif flags == cv2.EVENT_FLAG_CTRLKEY + cv2.EVENT_LBUTTONDOWN and len(points) < points_cnt:
                points.append((x, y))           

    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
    cv2.resizeWindow(window_name, 1080, 720)
    cv2.moveWindow(window_name, 0, 0)
    cv2.setMouseCallback(window_name, draw_circle)
    
    # Drawing section, scale drawing elements with image size
    circle_diam = int(0.003 * img_dimensions[1])
    lbl_offset = int(0.005 * img_dimensions[1])
    lbl_font_scale = (0.001 * img_dimensions[1])
    lbl_thickness = int(0.003 * img_dimensions[1])
    
    while 1:
        drawn_img = img.copy()
        
        for i, p in enumerate(points):
            cv2.circle(drawn_img, p, circle_diam , (0, 0, 255), cv2.FILLED)
            cv2.putText(drawn_img, str(i), (p[0] + circle_diam + lbl_offset, p[1] + circle_diam + lbl_offset),
                        0, lbl_font_scale, (0, 0, 255), lbl_thickness)
            
        cv2.imshow(window_name, drawn_img)
        k = cv2.waitKey(1) & 0xFF
        
        if k == ord('q'):
            break

    cv2.destroyAllWindows()
    
    if len(points) < 4: 
        raise ValueError('Number of choosen points must be >= 4.')
        
    return points

In [4]:
def copy_to(src, dst, mask):
    '''Python alternative to C++/Java OpenCV's Mat.copyTo().
    More: https://docs.opencv.org/trunk/d3/d63/classcv_1_1Mat.html#a626fe5f96d02525e2604d2ad46dd574f'''
    locs = np.where(mask != 0) # Get the non-zero mask locations
    dst[locs[0], locs[1]] = src[locs[0], locs[1]]
    return dst

def midpoint(ptA, ptB):
    '''Returns the midpoint between two input points.'''
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

def order_points(pts):
    '''Sorts the points based on their x-coordinates.'''
    xSorted = pts[np.argsort(pts[:, 0]), :]

    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (bl, tl) = leftMost

    # now that we have the top-left coordinate, use it as an
    # anchor to calculate the Euclidean distance between the
    # top-left and right-most points; by the Pythagorean
    # theorem, the point with the largest distance will be
    # our bottom-right point
    rightMost = rightMost[np.argsort(rightMost[:, 1]), :]
    (br, tr) = rightMost

    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    return np.array([tl, tr, br, bl], dtype="float32")

def rotateImage(image, angle, image_center=None):
    """ Rotates the input image by specified angle.
    
    Parameters
    ----------
    image : np.ndarray
        Image to be rotated.
    angle : float
        Rotation angle.
    image_center : Optional[tuple(int, int)]
        Center of rotation.
    Returns
    -------
    np.ndarray
        Returns the rotated input image by specified angle.
    """
    if image_center is None:
        image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

# TODO: Most inefficient function in my whole life 
def draw_rotated_text(img, text, point, angle, text_scale, text_color, text_thickness):
    img_filled = np.full(img.shape, text_color, dtype=np.uint8)
    # create rotated text mask
    text_mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    cv2.putText(text_mask, "{:.2f} cm".format(text), point, 0, text_scale, (255, 255, 255), text_thickness)
    if angle > 0:
        angle = -angle + 90
    elif angle < 0:
        angle = angle + 90
    text_mask = rotateImage(text_mask, -angle, point)
    result = copy_to(img_filled, img.copy(), text_mask)
    return result


def draw_real_sizes(img, rect, width_text, height_text, lbl_size_scale=2, lbl_color=(0, 0, 255), lbl_thickness=8):
    tl, tr, br, bl = order_points(cv2.boxPoints(rect))
    mid_pt_width = midpoint(tl, tr)
    mid_pt_height = midpoint(tr, br)
    
    # bottom-left points where labels are drawn
    pt_label_first =  (int(mid_pt_width[0] - 10), int(mid_pt_width[1] - 10))
    pt_label_second = (int(mid_pt_height[0] + 10), int(mid_pt_height[1]))
        
    result = draw_rotated_text(img, width_text, pt_label_first, rect[2], lbl_size_scale, lbl_color, lbl_thickness)
    result = draw_rotated_text(result, height_text, pt_label_second, rect[2], lbl_size_scale, lbl_color, lbl_thickness)
    return result

def show_scale(min_val, max_val, color_map='jet'):
    fig = plt.figure(figsize=(8, 3))
    ax1 = fig.add_axes([0.05, 0.80, 0.9, 0.15])
    norm = matplotlib.colors.Normalize(vmin=min_val, vmax=max_val)
    cb1 = matplotlib.colorbar.ColorbarBase(ax1, cmap=color_map, norm=norm, orientation='horizontal')
    cb1.set_label('Temperature °C')
    plt.show()

In [2]:
def select_image_points(img, points_cnt = 4):
    """ Opens a new image window, where user can interactively add or remove image points.
    Points are added while holding CTRL key + pressing left mouse button and removed by ALT key + pressing left mouse button.
    The point selection is terminated by pressing the 'q' key.
    
    Parameters
    ----------
    img : ndarray
        Input image where image points are choosen and drawn.
    points_cnt : Optional[int]
        A maximum number of points to choose. A minimum number of points to compute the projective transformation is 4.
    Returns
    -------
    list
        Returns a list of size >= 4 and size <= points_cnt such that each elements represent (x, y) coordinate in input image.
    """
    if points_cnt < 4: 
        raise ValueError('Number of points must be >= 4.')
        
    points = []
    window_name = 'Point selection'
    img_dimensions = img.shape[:2]
    pts_dist_thresh = 0.01 * img_dimensions[1] # Scale drawing elements with image size
    
    def draw_circle(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            if flags == cv2.EVENT_FLAG_ALTKEY + cv2.EVENT_LBUTTONDOWN: 
                for p in points:
                    if distance.euclidean(p, (x, y)) < pts_dist_thresh:
                        points.remove(p)
                        break
            elif flags == cv2.EVENT_FLAG_CTRLKEY + cv2.EVENT_LBUTTONDOWN and len(points) < points_cnt:
                points.append((x, y))           

    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
    cv2.resizeWindow(window_name, 1080, 720)
    cv2.moveWindow(window_name, 0, 0)
    cv2.setMouseCallback(window_name, draw_circle)
    
    # Drawing section, scale drawing elements with image size
    circle_diam = int(0.003 * img_dimensions[1])
    lbl_offset = int(0.005 * img_dimensions[1])
    lbl_font_scale = (0.001 * img_dimensions[1])
    lbl_thickness = int(0.003 * img_dimensions[1])
    
    while 1:
        drawn_img = img.copy()
        
        for i, p in enumerate(points):
            cv2.circle(drawn_img, p, circle_diam , (0, 0, 255), cv2.FILLED)
            cv2.putText(drawn_img, str(i), (p[0] + circle_diam + lbl_offset, p[1] + circle_diam + lbl_offset),
                        0, lbl_font_scale, (0, 0, 255), lbl_thickness)
            
        cv2.imshow(window_name, drawn_img)
        k = cv2.waitKey(1) & 0xFF
        
        if k == ord('q'):
            break

    cv2.destroyAllWindows()
    
    if len(points) < 4: 
        raise ValueError('Number of choosen points must be >= 4.')
        
    return points

In [3]:
def show_images(*imgs, scale=1, window_name='Image preview'):
    """ Opens multiple image previews depending on the length of the input *imgs list.
    The preview is terminated by pressing the 'q' key.
    
    Parameters
    ----------
    *imgs : list
        Multiple input images which have to be shown.
    scale : double
        Scale of shown image window.
    window_name : Optional[string]
        An optional window name.
    Returns
    -------
    None
    """
    def print_xy(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONUP:
            print('x = %d, y = %d'% (x, y)) 
            
    for i, img in enumerate(imgs, 1):
        h,w = img.shape[:2]
        window_name_id = window_name + ' ' + str(i)
        cv2.namedWindow(window_name_id, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
        cv2.resizeWindow(window_name_id, int(w * scale), int(h * scale))
        cv2.setMouseCallback(window_name_id, print_xy)
        cv2.moveWindow(window_name_id, (i-1)*int(w * scale), 0)

    while 1:
        for i, img in enumerate(imgs, 1):
            cv2.imshow(window_name + ' ' + str(i), img)
            
        k = cv2.waitKey(0)
        
        if k == ord('q') or k == 27:
            break

    cv2.destroyAllWindows()
    
def plot_images(*imgs):
    f = plt.figure(figsize=(16,9))
    for i, img in enumerate(imgs, 1):
        ax = f.add_subplot(1, len(imgs), i)
        ax.imshow(img[...,::-1])

<a id='ocr'>Metoda pro OCR obrazu</a>

In [9]:
def ocr(img_bin):
    '''Detects text in the file.
    
    Parameters
    ----------
    img_bin : numpy.ndarray
        Input binary image. White objects on black background.
    Returns
    -------
    Text on image.
    '''
    # Tesseract works with black objects on white background.
    img_bin = negative(img_bin)
    return pytesseract.image_to_string(Image.fromarray(img_bin))

<a id='others'>Metody ostatní</a>

In [10]:
def to_intensity(hue_angle):
    '''Converts color angle in HUE definition into intensity value of brightness image in opencv
    
    Parameters
    ----------
    hue_angle : int
        Angle in HUE definition.
    Returns
    -------
    Integer value that represents the same HUE value but in opencv brightness image.
    '''
    return int(hue_angle * 0.5)

<a id='cart_polar_functions'>Metody transformací</a>. Například převodu mezi kartézskými a polárními souřadnicemi nebo rotace.

In [11]:
# Linear polar warp help function
def polar_warp(img, full_radius=True, inverse=False):
    center = (img.shape[0]/2.0, img.shape[1]/2.0)
    
    if full_radius:
        radius = np.sqrt(((img.shape[0]/2.0)**2.0)+((img.shape[1]/2.0)**2.0))
    else:
        radius = center[0]
    
    method = cv2.WARP_FILL_OUTLIERS
    if inverse: 
        method += cv2.WARP_INVERSE_MAP
    dest = cv2.linearPolar(img, center, radius, method)
    return dest

In [12]:
def warp_to_cartesian(img, full_radius=True):
    return polar_warp(img, full_radius)

def warp_to_polar(img, full_radius=True):
    return polar_warp(img, full_radius, True)

def rotate(img, angle):
    height, width = img.shape[:2]
    image_center = (width/2, height/2)

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

    abs_cos = abs(rotation_mat[0,0])
    abs_sin = abs(rotation_mat[0,1])

    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    dest = cv2.warpAffine(img, rotation_mat, (bound_w, bound_h))
    return dest

<a id='artificial_funtions'>Metody na tvorbu umělých obrázků</a>

In [13]:
def artificial_circle_image(size):
    img_art_circ = np.ndarray((size, size), dtype=np.float32)
    step = 10
    for i in range(step, size, step):
        cv2.circle(img_art_circ, (int(size/2.0), int(size/2.0)), i-step, np.random.randint(0,255), thickness=4)
    return img_art_circ